In [45]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import numpy as np
import pickle

In [52]:
def moving_average(x, w):
        moving_avg = np.convolve(x, np.ones(w), 'valid') / w
        padding = np.full_like(np.empty(w), np.nan)
        return np.insert(moving_avg, 0, padding)

def moving_avg_diff(short, long):
        return (short - long) / long


def get_X_y(df, binary_output=True):
    
    min_period = 24*31 + 1
    
    percent_change = (df["Close"] - df["Close"].shift(1)) / df["Close"].shift(1)
    print(percent_change[:5])

    y = percent_change.copy()
    
    print(len(y))
    
    ## TEMP
    if binary_output == True:
        y = np.zeros(percent_change.shape)
        y[np.where(percent_change > 0)] = 1
    else:
        y = y.to_numpy()
    
    print(type(y))

    moving_average_1_hours = moving_average(df["Close"], 1)
    moving_average_2_hours = moving_average(df["Close"], 2)
    moving_average_3_hours = moving_average(df["Close"], 3)
    moving_average_12_hours = moving_average(df["Close"], 12)
    moving_average_24_hours = moving_average(df["Close"], 24)
    moving_average_48_hours = moving_average(df["Close"], 48)
    moving_average_1_week = moving_average(df["Close"], 24*7)
    moving_average_1_month = moving_average(df["Close"], 24*31)
    
    print(len(moving_average_1_month))
    
    mv_1h_2h = moving_avg_diff(moving_average_1_hours, moving_average_2_hours)
    mv_1h_3h = moving_avg_diff(moving_average_1_hours, moving_average_3_hours)
    mv_3h_12h = moving_avg_diff(moving_average_3_hours, moving_average_12_hours)
    mv_12h_24h = moving_avg_diff(moving_average_12_hours, moving_average_24_hours)
    mv_24h_48h = moving_avg_diff(moving_average_24_hours, moving_average_48_hours)
    mv_48h_1w = moving_avg_diff(moving_average_48_hours, moving_average_1_week)
    mv_1w_1m = moving_avg_diff(moving_average_1_week, moving_average_1_month)
    
    print(len(mv_1w_1m))
    
    X = np.stack((mv_1h_2h, mv_1h_3h, mv_3h_12h, mv_12h_24h, mv_24h_48h, mv_48h_1w, mv_1w_1m), axis=1)
    X = X[min_period:len(X)-1]
    y = y[min_period:]
    
    print(len(X), len(y))
    
    return (X, y)

In [56]:
df = pd.read_csv("../../data/clean/bitcoin_prices.csv")

df["Time"] = pd.to_datetime(df['Time'],unit='s')
df.set_index(pd.DatetimeIndex(df['Time']), inplace=True, drop=True)
start_date = datetime.datetime(2015,7,1)
df = df[df["Time"] > start_date]

offset = 24*31 + 1 # TODO: Refactor to output ox get_features
X, y = get_X_y(df)
df = df[offset:]
for i in range(0, len(X[0,:])):
    df["X_" + str(i)] = X[:,i]
df.describe()
print(df.columns)

Time
2015-07-20 21:00:00         NaN
2015-07-20 22:00:00    0.000504
2015-07-20 23:00:00    0.006687
2015-07-21 00:00:00    0.003179
2015-07-21 01:00:00   -0.001887
Name: Close, dtype: float64
47698
<class 'numpy.ndarray'>
47699
47699
46953 46953
Index(['Unnamed: 0', 'Time', 'Low', 'High', 'Open', 'Close', 'Volume', 'X_0',
       'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6'],
      dtype='object')


In [48]:
class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

In [49]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    data = bt.feeds.PandasData(dataname=df)
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00


In [90]:
# Create a Stratey
class NNStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        features = ["X_1", "X_2", "X_3", "X_4", "X_5", "X_6"]
        self.all_data = self.datas[0]
#         self.X = [self.datas[0][x] for x in features]

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        filename = "../../models/pickles/svm_binary_classifier"
        self.clf = pickle.load(open(filename, 'rb'))
#         # Add a MovingAverageSimple indicator
#         self.sma = bt.indicators.SimpleMovingAverage(
#             self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        for att in dir(self.datas[0]):
            print (att, getattr(self.datas[0],att))
        print(self.datas[0].close)
#         print(self.all_data)
#         print(self.all_data[0])
#         print(self.datas[0]["X_1"])
        print(self.datas[0].X_1)
        X = [self.datas[0][x] for x in features]
        print(X)
        pred = self.clf.predict(X)
        print(pred)
        if not self.position:
            if pred == 1:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        else:
            self.log('SELL CREATE, %.2f' % self.dataclose[0])
            self.sell()

#         # Check if we are in the market
#         if not self.position:

#             if self.dataclose[0] > self.sma[0]:

#                 # BUY, BUY, BUY!!! (with all possible default parameters)
#                 self.log('BUY CREATE, %.2f' % self.dataclose[0])

#                 # Keep track of the created order to avoid a 2nd order
#                 self.order = self.buy()

#         else:

#             if self.dataclose[0] < self.sma[0]:
#                 # SELL, SELL, SELL!!! (with all possible default parameters)
#                 self.log('SELL CREATE, %.2f' % self.dataclose[0])

#                 # Keep track of the created order to avoid a 2nd order
#                 self.order = self.sell()

class PandasData(bt.feed.DataBase):
    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
#         ('volume', -1),
#         ('openinterest', -1),
        ('X_1', -1),
        ('X_2', 7)
        
    )


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(NNStrategy)

    data = bt.feeds.PandasData(dataname=df)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(1000)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=0.01)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)
    cerebro.broker.addcommissioninfo(CommInfoFractional())

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000.00
2015-08-20, Close, 236.60
CONNBROKEN 2
CONNECTED 0
Close 0
DELAYED 3
DISCONNECTED 1
DateTime 6
High 2
IndType 0
LIVE 4
LineOrder [6, 3, 2, 1, 0, 4, 5]
Low 1
NOTSUBSCRIBED 5
NOTSUPPORTED_TF 6
ObsType 2
Open 3
OpenInterest 5
StratType 1
UNKNOWN 7
Volume 4
_NOTIFNAMES ['CONNECTED', 'DISCONNECTED', 'CONNBROKEN', 'DELAYED', 'LIVE', 'NOTSUBSCRIBED', 'NOTSUPPORTED_TIMEFRAME', 'UNKNOWN']
_OwnerCls None
__abs__ <bound method LineRoot.__abs__ of <backtrader.feeds.pandafeed.PandasData object at 0x7f93a131d370>>
__add__ <bound method LineRoot.__add__ of <backtrader.feeds.pandafeed.PandasData object at 0x7f93a131d370>>
__bool__ <bound method LineRoot.__nonzero__ of <backtrader.feeds.pandafeed.PandasData object at 0x7f93a131d370>>
__call__ <bound method LineSeries.__call__ of <backtrader.feeds.pandafeed.PandasData object at 0x7f93a131d370>>
__class__ <class 'backtrader.feeds.pandafeed.PandasData'>
__delattr__ <method-wrapper '__delattr__' of PandasData object at 0x7

AttributeError: 'Lines_LineSeries_DataSeries_OHLC_OHLCDateTime_Abst' object has no attribute 'X_1'

In [ ]:
cerebro.plot()